# Online Performance Prediction Simulation

In [ ]:
import numpy as np
import torch
from pickle import load
import matplotlib.pyplot as plt

# custom modules
from method import compute_eval_matrices
from watercan import GroupWaterCan
from visualization import make_perform_video

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Get the data

In [ ]:
dataset_name='uav_data'#'N-CMAPSS' #
threshold_name=''#'-0.1' #  '+0.1' #

# Load Test data

In [ ]:
with open(f'./data/{dataset_name}/performs_info.pkl', 'rb') as file:
        performs_info=load(file)

with open(f'./data/{dataset_name}/times_test{threshold_name}.pkl', 'rb') as file:
        t_observ = load(file)

with open(f'./data/{dataset_name}/times_test{threshold_name}_threshold.pkl', 'rb') as file:
        t_observ_thresholds = load(file)


performs = {}
thresholds = {}
for name in performs_info.keys():
        with open(f'./data/{dataset_name}/{name}_test{threshold_name}.pkl', 'rb') as file:
                performs[name] = load(file)
        with open(f'./data/{dataset_name}/{name}_test{threshold_name}_threshold.pkl', 'rb') as file:
                thresholds[name]= load(file)

## Choose Library

In [ ]:
library_address=f'./Library/MTL_2024-08-17_{dataset_name}/'

## Load Group Distributions

In [ ]:
group_distributions={}
for perform_name in performs_info.keys():
    perform_address=library_address+f'{perform_name}/'
    configs=torch.load(perform_address+'config.pth')
    model_state=torch.load(perform_address+'model_state.pth')
    model=GroupWaterCan(configs)
    model.load_state_dict(model_state)
    model.to(device)
    group_distributions[perform_name]=model
t_observ_torch=[torch.tensor(time,device=device,dtype=torch.float32).unsqueeze(dim=1) for time in t_observ]

# Compute and prepare A,b and time

In [ ]:
n_test_unit=len(t_observ)
As,Ss=compute_eval_matrices(group_distributions,t_observ_torch)
test_unit_list=range(n_test_unit)
bs = performs

In [ ]:
n_train_unit =20 # 20
n_paths = 10 # 10 #
gammas={'SOC': 100, #0.001
        'POS':100, #0.1 
        'CUM':100}
ood_coef=1.
time_unit='hours'

In [ ]:
performs_info.keys()

In [ ]:
unit=1
end_time=80
density_time=200
t = np.linspace(0, end_time, density_time)

for name in performs_info.keys():#
        threshold=np.interp(t, t_observ_thresholds[unit], thresholds[name][unit])

        y_lim=performs_info[name]['y_lim']
        loc=performs_info[name]['loc']
        gamma=gammas[name] 
        group_distribution=group_distributions[name].get_sub_watercan(range(n_train_unit))
        A=As[name][unit][:,:n_train_unit]
        b=bs[name][unit]
        S=Ss[name][unit][:,:n_train_unit]
        make_perform_video(unit,group_distribution, 
                        t,threshold, t_observ[unit], 
                        A,b,S,
                        time_est=None, A_est=None, b_est=None, 
                        gamma=gamma,n_train_unit=n_train_unit, n_paths=n_paths,ood_coef=ood_coef, 
                        name=name,frames=20,  
                        y_lim=y_lim, loc=loc, time_unit=time_unit,save=False)
